- Code ref : https://www.kaggle.com/code/ritesh1420/image-caption-generation-using-trasformer

### 1. Data understanding
---
1.1 Import the dataset and read image & captions into two seperate variables

1.2 Visualise both the images & text present in the dataset

1.3 Create word-to-index and index-to-word mappings.

1.4 Create a dataframe which summarizes the image, path & captions as a dataframe

1.5 Visualise the top 30 occuring words in the captions

1.6 Create a list which contains all the captions & path